In [26]:
import pandas as pd
# import ydata_profiling  as ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix



df = pd.read_csv('D:\Hutson\learning-materials\AI&ML\Khóa 12-02AIMLDLCV nâng cao\Class\Datasets\StudentScore.xls')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [27]:
df['parental level of education'].unique()

array(["bachelor's degree", 'some college', "master's degree",
       "associate's degree", 'high school', 'some high school'],
      dtype=object)

In [28]:
numeric_data = df.select_dtypes(include=[int, float])
df[numeric_data.columns].corr()  

,math score,reading score,writing score
math score,1.000000,0.817580,0.802642
reading score,0.817580,1.000000,0.954598
writing score,0.802642,0.954598,1.000000


In [29]:
target = 'math score'

x = df.drop(target, axis=1)
y = df[target]

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [31]:
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

In [32]:
education_values = ["some high school", "high school", "some college", "associate's degree", "bachelor's degree", "master's degree"]
gender_values = ['male','female']
lunch_values = x_train['lunch'].unique()
test_preparation_course_values = x_train['test preparation course'].unique()


ord_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', OrdinalEncoder(categories=[education_values,gender_values,lunch_values,test_preparation_course_values]))
])

In [33]:
nom_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', OneHotEncoder())
])

In [34]:
preprocessor = ColumnTransformer(transformers=[
    ("num_features", num_transformer, ["reading score", "writing score"]),
    ("nom_features", nom_transformer,["race/ethnicity"]),
    ("ord_features", ord_transformer,["parental level of education","gender","lunch","test preparation course"])]
    )

In [35]:
reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', LinearRegression())])

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

for i, j in zip(y_pred, y_test):
    print("Predicted: ", i, "Actual: ", j)

Predicted:  76.38580410731748 Actual:  91
Predicted:  58.25576114220228 Actual:  53
Predicted:  77.28873659169835 Actual:  80
Predicted:  76.28810374839044 Actual:  74
Predicted:  87.05386270531177 Actual:  84
Predicted:  79.00478630851018 Actual:  81
Predicted:  64.59569129545402 Actual:  69
Predicted:  53.492521598971265 Actual:  54
Predicted:  74.20133979837443 Actual:  87
Predicted:  49.7072978797995 Actual:  51
Predicted:  52.843652786208544 Actual:  45
Predicted:  37.26601145120831 Actual:  30
Predicted:  67.75769460401824 Actual:  67
Predicted:  51.66046841613638 Actual:  49
Predicted:  86.13587826442664 Actual:  85
Predicted:  71.01746367338335 Actual:  65
Predicted:  51.890489591525146 Actual:  53
Predicted:  47.12540851335166 Actual:  55
Predicted:  49.33518376760799 Actual:  48
Predicted:  52.68499407813897 Actual:  56
Predicted:  74.46916540569312 Actual:  79
Predicted:  39.16305491119476 Actual:  43
Predicted:  56.76747808101762 Actual:  45
Predicted:  36.23899962271891 Ac

In [36]:
### r2 score is most common metric for regression (the higher the better)
print(reg.score(x_test, y_test))

0.8815597679452446
